# Experimental design

The objective of this little program is to, in every experimental data, upload the data, and, later, obtain the values of the flowRate, the vacuum level and its graphics. 

In [1]:
from   fluids               import *
from   thermo               import Mixture
from   scipy.constants      import atmosphere, zero_Celsius, mmHg, g
import scipy                as     sci
import uncertainties 
from   uncertainties        import *
from   uncertainties.unumpy import *
import uncertainties        as     unumpy
from   numpy                import *
from   pandas               import *
import matplotlib.pyplot    as     plt

In [2]:
# For the calculation of flow rate with uncertainties:
wrappedFlowMeter = uncertainties.wrap( differential_pressure_meter_solver) 

## Functions definition
This functions calculates flow rate from a long radious nozzle. Which in our case it means: the secondary flow or the aspiration flow 

In [3]:
def set_of_const  (P_amb , T_amb):
            
    P_amb     = P_amb * mmHg           # Pressure in Pa
    T_amb     = T_amb + zero_Celsius   # T in K 

# Here we calculate all what it is important with the air at the ambient pressure
    
    air       = Mixture( 'air', T = T_amb , P = P_amb)
    k         = air.isentropic_exponent
    mu        = air.mu
    rho       = air.rho
    
    return P_amb, T_amb, k , mu , rho

In [4]:
def flow_meter_calculator (mesures1 , mesures2 , manRelRho, deltaH , Do , D1 , rho , k , mu  ):  

# Experimental setup
    taps                            = 'flange'
    meter_type                      = 'long radius nozzle'
    
# Inci tractament de les dades
    flow_meter_calculator           = DataFrame()
    flow_meter_calculator['h1']     = uarray(mesures1 ,DeltaH*ones(len (mesures1)))
    flow_meter_calculator['h2']     = uarray(mesures2 ,DeltaH*ones(len (mesures2)))
    
# ManRelRho is the relative density of the manometric fluid
    flow_meter_calculator['p1']     = flow_meter_calculator.h2 * g * manRelRho # p1 is the bigger pressure, and 
    flow_meter_calculator['p2']     = flow_meter_calculator.h1 * g * manRelRho # p2 is the low pressure in the nozzle
    flow_meter_calculator['DeltaP'] = flow_meter_calculator.p1 - flow_meter_calculator.p2

# First we have to initialize the vector
    mdot_s_1                        = uarray( zeros(len (mesures1)) , zeros(len (mesures1)) )

    for i,row in flow_meter_calculator.iterrows():
        
        if nominal_value(row.DeltaP) != 0.0:             # Just to avoid division by 0 when DeltaP=0. .
            P2                      = P_amb - row.DeltaP
            mdot_s_1[i]             = wrappedFlowMeter(D =D1, D2 =Do, P1 =P_amb, P2 =P_amb-row.DeltaP, rho=rho, mu=mu, k=k, meter_type=meter_type, taps=taps)
        else:
            mdot_s_1[i]             = ufloat(0.0,0.0)

    flow_meter_calculator['mdot_s'] = mdot_s_1
    flow_meter_calculator['QN_s']   = flow_meter_calculator.mdot_s * 60000/rhoN #flow_meter_calculator.mdot_s*60000/rhoN 
    
    return flow_meter_calculator

This function calculates the pressure in the vessel. It uses mercury.

In [5]:
def pressure_v  (pressure_vessel, mesures1, mesures2, DeltaH_Hg , P_amb):
    
#     pressure_vessel                 = DataFrame()
    pressure_vessel['h3']           = uarray( mesures1 , DeltaH_Hg * ones(len (mesures1)) )
    pressure_vessel['h4']           = uarray( mesures2 , DeltaH_Hg * ones(len (mesures2)) )
    pressure_vessel['p3']           = pressure_vessel.h3 * mmHg  #mmHg that it is been taken from scipy.constants
    pressure_vessel['p4']           = pressure_vessel.h4 * mmHg
    pressure_vessel['P_vessel']     = pressure_vessel.p3 - pressure_vessel.p4
    pressure_vessel['P_vessel_dim'] = (P_amb - pressure_vessel.P_vessel) /P_amb
    pressure_vessel['P_vess_AR']    = 100 - pressure_vessel.P_vessel_dim*100           # Aquí ho transportem al percentatge normal, per AR


Here it is calculated the flow consumtion

In [6]:
def mdot_f ( mdot_p , mesures1 , mesures2 , DeltaP , DeltaT , mesuresT , D1 , D2 ):

    # Experimental setup
    taps              = 'flange'
    meter_type        = 'ISO 5167 orifice'  # meter_type = 'ISA 1932 nozzle'

# First we have to initialize the vector   
    mdot              = uarray( zeros(len (mesures1)) , zeros(len (mesures1)) )  

# Here we inicialize the vectors with their uncertainties
    p1                = uarray( mesures1 ,   DeltaP * ones(len (mesures1)) )
    p2                = uarray( mesures2 ,   DeltaP * ones(len (mesures2)) )
    T1                = uarray( mesuresT ,   DeltaT * ones(len (mesuresT)) )
# Here we start to carry the units as we need them     
    T1                = T1 + zero_Celsius    # Temperature in KELVIN
    p1                = ( p1 + 1 ) * 1e5     # Pressió en Pa, absoluts
    p2                = ( p2 + 1 ) * 1e5

    for i in range( len(mesures1) ):
# l'aire es calcula amb cada valor de P i de T. IMPORTANT: el nominal_value aconsegueix fer-hi passar només el valor correcte.
        air           = Mixture( 'air' , T = float(nominal_value (T1[i]) ) , P = float( nominal_value( p1[i]) ) )
        r             = sci.constants.R / air.MW * 1000
        k             = air.isentropic_exponent
        mu            = air.mu
        rho           = p1[i] / ( r * T1[i] )    
#  Here is where we write the vector mdot
        mdot[i]       = wrappedFlowMeter(D=D1, D2=D2, P1=p1[i], P2=p2[i], rho=rho, mu=mu, k=k, meter_type=meter_type, taps=taps)

    mdot_p['mdot_p']  = mdot
    mdot_p['QN_p']    = mdot_p.mdot_p * 60000 / rhoN
    mdot_p['mu']      = mdot_p.QN_s / mdot_p.QN_p

Here I define the function that we will use for the numerical results

In [7]:
def numDat    (mdot_s , mdot_p, p0_s , errNumSec , errNumPri ):

#Here we first have to inicialize the variables. In this case has to be multiplied by 360/5=72 
    mdot_s               = mdot_s*72
    mdot_p               = mdot_p*72
    mdot_f_s             = uarray( mdot_s , mdot_s * errNumSec)
    mdot_f_p             = uarray( mdot_p , mdot_p * errNumPri)

    # First we have to create the panda dataFrame
    numDat               = DataFrame()
    numDat['p0_s_dim']   = p0_s                     # These are the total pressure in the vessel
    numDat['mdot_s']     = mdot_f_s
    numDat['mdot_p']     = mdot_f_p  
    numDat['p0_s_AR']    = 100 - numDat.p0_s_dim*100
    
# We transform the calculations into Nl/min
    numDat['QN_s']       = numDat.mdot_s*60000/rhoN
    numDat['QN_p']       = numDat.mdot_p*60000/rhoN
# Finally we calculate the mu
    numDat['mu']         = numDat.QN_s / numDat.QN_p

    return numDat     

## Calculation of flow rate. In order of experiment
In the simulations we will call the aspiration flow rate as secondary flow rate. <br>
In the simulations we will call the consumption flow as the primary flow rate <br>
We will use the same but now for the value of the comsumption flow. Also , density is function of pressure, since here air is compressed. 
<br> Meter_type changes to ISO 5167, because in this measure, the nozzle is inside the pipe <br>
Uncertainties and flow meter from the fluid package is used <br> <br> First of all we start to define our geometry of working

In [8]:
# Uncertainties
# Here I put all the uncertainties together

DeltaH                    = 5           # uncertainty in pressure,in mm. abans n'hi havia 2
DeltaH_Hg                 = 5           # uncertainty for mercury
DeltaP                    = 0.01        # uncertainty in pressure, in bar
DeltaT                    = 0.1         # uncertainty in kelvin
DeltaMM                   = 20e-5        #uncertainty of measures of mm, expresed in meters

# Here I put all the measures of the pipe and the diameter of the nozzle 

D1                        = ufloat( 22e-3   , DeltaMM)      # pipe diameter, for the large nozzle pressure in m
D2                        = ufloat( 25e-3   , DeltaMM)      # pipe diameter, for the small nozzle pressure in m
D3                        = ufloat( 22e-3   , DeltaMM)      # pipe diameter, m'imagino q és el diàmetre de la canonada
Do_1                      = ufloat( 10.3e-3 , DeltaMM)      # orifice diameter in m (large nozzle)
Do_2                      = ufloat( 5.15e-3 , DeltaMM)      # orifice diameter in m (small nozzle)
Do_3                      = ufloat( 4.2e-3  , DeltaMM)      # sospito q és el diàmetre del nozzle del diafragma q calcula el consum

First definitions 

In [9]:
airN                      = Mixture('air',T=293,P=101325)
rhoN                      = airN.rho

### First experiment 
<br>Date:            XX/xx/18          
Nozzle:          Large nozzle
<br>Fluid_secondary: Water
<br>Made by:         Robert 

In [10]:
# First, definition of experimental conditions
P_amb            = 745                  # Ambient pressure in mmHg
T_amb            = 25                   # T in degrees
# here we call the constants function in order to get our constants correctly according 
P_amb, T_amb, k , mu , rho = set_of_const (P_amb , T_amb)
### The 11th measurement seems to be wrong
mesures1         = [226,248,282,318,330,360,391,413,437,459,470]
mesures2         = [717,694,659,624,613,582,551,529,504,484,470]
manRelRho        = 1 #It is water
largeNozzleData  = flow_meter_calculator (mesures1 , mesures2 , manRelRho, DeltaH , Do_1 , D1 , rho , k , mu  )

mesures1         = [1176,1187,1211,1233,1248,1267,1296,1334,1363,1410,1463]
mesures2         = [1143,1131,1109,1087,1072,1053,1024,988,960,913,860]
pressure_v (largeNozzleData,mesures1 , mesures2, DeltaH_Hg, P_amb )

mesures1        = [6.72,6.72,6.71,6.72,6.70,6.70,6.70,6.69,6.69,6.69,6.69]
mesures2        = [6.00,6.01,6.00,6.01,6.00,6.00,6.00,6.00,6.00,5.99,6.00]
mesuresT        = [18.0,18.0,18.1,18.2,19.1,19.1,19.2,19.3,19.5,19.6,19.6]
mdot_f ( largeNozzleData, mesures1, mesures2, DeltaP , DeltaT , mesuresT , D3 , Do_3 )

###  Second experiment

<br>Date:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;XX/xx/18          
Nozzle: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Small nozzle
<br>Fluid_secondary: Water
<br>Made by:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Robert 

In [11]:
# First, definition of experimental conditions
P_amb     = 750                  # Ambient pressure in mmHg
T_amb     = 25                   # T in degrees

P_amb, T_amb, k , mu , rho = set_of_const (P_amb , T_amb)
# Entada dels valors de les lectures
mesures1                  = [ 411,373,331,281,221,169,124,78,15 ]
mesures2                  = [ 509,548,590,642,702,752,799,843,901 ]
manRelRho                 = 1   # It is water
# Here we calculate 
smallNozzleData1          = flow_meter_calculator (mesures1 , mesures2 , manRelRho, DeltaH ,Do_2 , D2 , rho , k , mu  )

mesures1                  = [799,784,774,767,757,749,743,739,734]
mesures2                  = [227,240,252,256,268,276,281,286,290]

pressure_v (smallNozzleData1, mesures1 , mesures2, DeltaH_Hg, P_amb )

#dades 2a mesura (small nozzle)
mesures1                  = [6.67,6.67,6.67,6.69,6.67,6.70,6.72,6.67,6.67]
mesures2                  = [6.03,6.05,6.03,6.06,6.02,6.07,6.05,6.05,6.04]
T1                        = [24.5,24.6,24.6,24.7,24.8,24.9,25.0,25.1,25.0]
mdot_f ( smallNozzleData1, mesures1, mesures2, DeltaP , DeltaT , T1 ,D3 , Do_3 )

### Third experiment

<br>Date:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;30/11/18          
Nozzle: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Small nozzle
<br>Fluid_secondary: Alcohol
<br>Made by:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;LMacia

In [12]:
# First, definition of experimental conditions
P_amb     = 739
# Ambient pressure in mmHg
T_amb     = 25                   # T in degrees les lectures
P_amb, T_amb, k , mu , rho = set_of_const (P_amb , T_amb)
# Entada dels valors de les lectures
mesures1         = [249,247,213,227,220,205,153,150,90]
mesures2         = [251,254,279,254,261,290,342,345,403]
manRelRho        = 0.7918   # It is alcohol
smallNozzleData2 = flow_meter_calculator (mesures1 , mesures2 , manRelRho, DeltaH , Do_2 , D2 , rho , k , mu  )

mesures1         = [810,811,799,805,801,793,787,779,775]
mesures2         = [218,211,221,218,221,228,236,243,248]
pressure_v (smallNozzleData2, mesures1 , mesures2, DeltaH_Hg, P_amb )

mesures1         = [6.67,6.67,6.67,6.69,6.67,6.70,6.72,6.67,6.67]
mesures2         = [6.03,6.05,6.03,6.06,6.02,6.07,6.05,6.05,6.04]
T1               = [24.9,25.0,24.6,24.7,24.8,24.9,25.0,25.1,25]

mdot_f ( smallNozzleData2, mesures1, mesures2, DeltaP , DeltaT , T1 , D3 , Do_3 )

Càlcul de la K

## Numerical results obtained with the two solvers

First we introduce the errors considered that we have in the flow Rates

In [13]:
errNumSec = 0.004
errNumPri = 0.003

### HiSA

In [14]:
# These are the total pressure in the vessel
p0_s                 = array([100,80,60,40,30,27.5,25,21.73])/100
mdot_s               = array([1.30,1.03,0.73,0.45,0.33,0.26,0.14,0.0])*1e-4
mdot_p               = array([1.315,1.314,1.314,1.314,1.314,1.314,1.314,1.314])*1e-4

numData              = numDat( mdot_s , mdot_p, p0_s , errNumSec , errNumPri )

### rhoCentralFoam

In [15]:
p0RCF_s                = array([100,80,70,60,50,40,30,20]) /100
#This is the calculated flow rate for variable ps
mdotRCF_s              = array([1.21,0.94,0.81,0.69,0.56,0.44,0.35,0.18]) *1e-4
mdotRCF_p              = array([1.31,1.31,1.31,1.31,1.31,1.31,1.31,1.31])*1e-4

numDataRCF             = numDat( mdotRCF_s ,mdotRCF_p, p0RCF_s , errNumSec , errNumPri )

## Graphics

In [16]:
# Gràcies a aquest widget ens permet veure les gràfiques millor
%matplotlib widget     

fig  = plt.figure()               # Aquí es crea la instància de la immatge
fig1 = fig.add_subplot(1,1,1)     # aquí li indiquem que n'hi haurà una

fig1.errorbar(y    = nominal_values(smallNozzleData1.QN_s) , x     = nominal_values(smallNozzleData1.P_vess_AR),\
              yerr = std_devs(smallNozzleData1.QN_s)       , xerr  = std_devs(smallNozzleData1.P_vess_AR),\
              fmt  = 's'                                   , label = "Experimental Small nozzle 1")

fig1.errorbar( y   = nominal_values(smallNozzleData2.QN_s) , x     = nominal_values(smallNozzleData2.P_vess_AR),\
              yerr = std_devs(smallNozzleData2.QN_s)       , xerr  = std_devs(smallNozzleData2.P_vess_AR),\
              fmt  = 's'                                   , label = "Experimental Small nozzle 2")

fig1.errorbar( y   = nominal_values(largeNozzleData.QN_s)  ,  x    = nominal_values(largeNozzleData.P_vess_AR),\
              yerr = std_devs(largeNozzleData.QN_s)        ,  xerr = std_devs(largeNozzleData.P_vess_AR),\
              fmt  = 'o'                                   , label = "Experimental Large nozzle")

fig1.errorbar( y   = nominal_values(numData.QN_s)          , x     = nominal_values(numData.p0_s_AR),\
              yerr = std_devs(numData.QN_s)                , xerr  = std_devs(numData.p0_s_AR),\
              fmt  = 'v'                                   , label ="Numerical")

fig1.errorbar( y   = nominal_values(numDataRCF.QN_s)       , x     = nominal_values(numDataRCF.p0_s_AR), \
              yerr = std_devs(numDataRCF.QN_s)             , xerr  = std_devs(numDataRCF.p0_s_AR),\
              fmt  = 'v'                                   , label = "Numerical RCF")

plt.xlabel("Experimental Results vs Numerical Results ")

# plt.xticks(np.arange(0.18, 0.4, 0.1))
# plt.yticks(np.arange(  min(nominal_value(largeNozzleData.QN_s)), max(nominal_value(largeNozzleData.QN_s)), 0.1))
# plt.xticks(np.arange(min(nominal_values(largeNozzleData.P_vessel_dim)), max(nominal_values(largeNozzleData.P_vessel_dim)), 0.1))

# plt.gca().invert_xaxis()   # Gràcies a aquesta comanda m'ha permes girar l'eix X i aconseguir veure-ho com ho vol veure el sergi
fig1.set_xlabel('Depression [%]')
fig1.set_ylabel('NL/min')
axes = plt.gca()

# axes.set_xlim([ 70 , 83 ])
# axes.set_ylim([ -2 , 100 ])

plt.grid()                 # Ara ens mostrarà les gràfiques 
fig1.legend()              # Mostra la llegenda

FigureCanvasNbAgg()

In [24]:
mufig  = plt.figure()
mufig1 = mufig.add_subplot(1,1,1)

mufig1.errorbar( x  = nominal_values(smallNozzleData1.mu) , y     = nominal_values(smallNozzleData1.P_vessel_dim),\
               xerr = std_devs(smallNozzleData1.mu)       , yerr  = std_devs(smallNozzleData1.P_vessel_dim),\
               fmt  ='s'                                  , label ="Experimental Small nozzle 1")

mufig1.errorbar( x = nominal_values(smallNozzleData2.mu)  , y     = nominal_values(smallNozzleData2.P_vessel_dim),\
              xerr = std_devs(smallNozzleData2.mu)        , yerr  = std_devs(smallNozzleData2.P_vessel_dim),\
              fmt  ='s'                                   , label = "Experimental Small nozzle 2")

mufig1.errorbar( x = nominal_values(largeNozzleData.mu)   , y     = nominal_values(largeNozzleData.P_vessel_dim),\
              xerr = std_devs(largeNozzleData.mu)         , yerr  = std_devs(largeNozzleData.P_vessel_dim),\
              fmt  ='4'                                   , label = "Experimental Large nozzle")

mufig1.errorbar( x  = nominal_values(numData.mu)          , y     = nominal_values(numData.p0_s_dim),\
              xerr  = std_devs(numData.mu)                , yerr  = std_devs(numData.p0_s_dim),\
                fmt = 'v'                                 , label = "Numerical HiSA")

mufig1.errorbar( x = nominal_values(numDataRCF.mu)        , y     = nominal_values(numDataRCF.p0_s_dim),\
              xerr = std_devs(numDataRCF.mu)              , yerr  = std_devs(numDataRCF.p0_s_dim),
                fmt = 'v'                                 , label ="Numerical RCF")

mufig1.set_xlabel('$\mu$')
mufig1.set_ylabel('$P_s^*$')

mufig1.legend()
plt.grid() 

# plt.gca().invert_xaxis()
# mufig.savefig(fname="mu_vs_p_ast.pdf")

FigureCanvasNbAgg()

In [18]:
fig2 = plt.figure()
fig22 = fig2.add_subplot(1,1,1)

fig22.errorbar( x   = nominal_values(smallNozzleData1.QN_s) , y     = nominal_values(smallNozzleData1.P_vessel_dim),\
              xerr = std_devs(smallNozzleData1.QN_s)       , yerr  = std_devs(smallNozzleData1.P_vessel_dim),\
              fmt  ='s'                                    , label = "Experimental Small nozzle 1")
 
fig22.errorbar( x   = nominal_values(smallNozzleData2.QN_s) , y     = nominal_values(smallNozzleData2.P_vessel_dim),\
              xerr = std_devs(smallNozzleData2.QN_s)       , yerr  = std_devs(smallNozzleData2.P_vessel_dim),\
              fmt  = 's'                                   , label = "Experimental Small nozzle 2")

fig22.errorbar( x   = nominal_values(largeNozzleData.QN_s) , y     = nominal_values(largeNozzleData.P_vessel_dim),\
              xerr = std_devs(largeNozzleData.QN_s)       , yerr  = std_devs(largeNozzleData.P_vessel_dim),\
              fmt  = 'o'                                  , label ="Experimental Large nozzle")

plt.xlabel("first results")
plt.grid() 
plt.gca().invert_xaxis()
axes = plt.gca()
axes.set_xlim([ 500 , 400])
fig22.legend()

FigureCanvasNbAgg()

### This is the way to pass a pandas object (with uncertainty) to another program through a file
With the "pickle" method. See https://docs.python.org/3/library/pickle.html

In [19]:
import pickle

expNozzleData = pd.concat( [ smallNozzleData2 , smallNozzleData1 , largeNozzleData ] , ignore_index = 'True' )

with open('mdotLABSONvsPressure.pickle', 'wb') as f:
    
    pickle.dump( expNozzleData , f , pickle.HIGHEST_PROTOCOL )       # Pickle the 'data' dictionary using the highest protocol available.

In [20]:
fig3 = plt.figure()
fig33 = fig3.add_subplot(1,1,1)
# wrappedGradient = uncertainties.wrap( np.gradient) 
largeNozzleData  ['der']    = np.gradient(nominal_values( largeNozzleData.P_vessel_dim) , nominal_values( largeNozzleData.QN_s))
smallNozzleData1 ['der']    = np.gradient(nominal_values(smallNozzleData1.P_vessel_dim) , nominal_values(smallNozzleData1.QN_s))
smallNozzleData2 ['der']    = np.gradient(nominal_values(smallNozzleData2.P_vessel_dim) , nominal_values(smallNozzleData2.QN_s))


fig33.errorbar( x   = nominal_values(smallNozzleData1.QN_s) , y     = nominal_values(smallNozzleData1.der),\
               xerr = std_devs(smallNozzleData1.QN_s)       , yerr  = std_devs(smallNozzleData1.der),\
               fmt  ='s'                                    , label = "Experimental Small nozzle 1")
 
fig33.errorbar( x   = nominal_values(smallNozzleData2.QN_s) , y     = nominal_values(smallNozzleData2.der),\
                xerr = std_devs(smallNozzleData2.QN_s)       , yerr  = std_devs(smallNozzleData2.der),\
                fmt  = 's'                                   , label = "Experimental Small nozzle 2")
 
fig33.errorbar( x   = nominal_values(largeNozzleData.QN_s) , y     = nominal_values(largeNozzleData.der),\
              xerr = std_devs(largeNozzleData.QN_s)       , yerr  = std_devs(largeNozzleData.der),\
              fmt  = 'o'                                  , label ="Experimental Large nozzle")

# fig33.errorbar( x   = nominal_values(largeNozzleData.QN_s) , y     = nominal_values(largeNozzleData.P_vessel_dim),\
#                xerr = std_devs(largeNozzleData.QN_s)       , yerr  = std_devs(largeNozzleData.P_vessel_dim),\
#                fmt  = 'o'                                  , label ="Experimental Large nozzle")

plt.xlabel("Derivada de P respecte Q")
plt.grid() 
plt.gca().invert_xaxis()
fig33.legend()

FigureCanvasNbAgg()

In [21]:
print (numData)

   p0_s_dim               mdot_s               mdot_p  p0_s_AR          QN_s  \
0    1.0000    0.00936+/-0.00004  0.009468+/-0.000028     0.00   466.0+/-1.9   
1    0.8000  0.007416+/-0.000030  0.009461+/-0.000028    20.00   369.2+/-1.5   
2    0.6000  0.005256+/-0.000021  0.009461+/-0.000028    40.00   261.7+/-1.0   
3    0.4000  0.003240+/-0.000013  0.009461+/-0.000028    60.00   161.3+/-0.6   
4    0.3000  0.002376+/-0.000010  0.009461+/-0.000028    70.00   118.3+/-0.5   
5    0.2750  0.001872+/-0.000007  0.009461+/-0.000028    72.50    93.2+/-0.4   
6    0.2500  0.001008+/-0.000004  0.009461+/-0.000028    75.00  50.18+/-0.20   
7    0.2173              0.0+/-0  0.009461+/-0.000028    78.27       0.0+/-0   

          QN_p               mu  
0  471.4+/-1.4    0.989+/-0.005  
1  471.0+/-1.4    0.784+/-0.004  
2  471.0+/-1.4  0.5556+/-0.0028  
3  471.0+/-1.4  0.3425+/-0.0017  
4  471.0+/-1.4  0.2511+/-0.0013  
5  471.0+/-1.4  0.1979+/-0.0010  
6  471.0+/-1.4  0.1065+/-0.0005  
7  471.

In [22]:
print(np.std(largeNozzleData.der),np.std(smallNozzleData1.der))

0.00043018155388950373 0.000339003144009573


In [23]:
fig4 = plt.figure()
fig44 = fig4.add_subplot(1,1,1)
# wrappedGradient = uncertainties.wrap( np.gradient) 
numData    ['der']  = np.gradient(nominal_values(numData.p0_s_AR) , nominal_values( numData.mdot_s))
# numDataRCF ['der']  = np.gradient(nominal_values(numDataRCF.p0_s_AR) , nominal_values(numDataRCF.QN_s))


fig44.errorbar( x   = nominal_values(numData.mu) , y     = nominal_values(numData.der),\
               xerr = std_devs(numData.mu)      , yerr  = std_devs(numData.der),\
               fmt  ='s'                                    , label = "Numerical HiSA ")
 
# fig33.errorbar( x   = nominal_values(smallNozzleData2.QN_s) , y     = nominal_values(smallNozzleData2.der),\
#                xerr = std_devs(smallNozzleData2.QN_s)       , yerr  = std_devs(smallNozzleData2.der),\
#                fmt  = 's'                                   , label = "Experimental Small nozzle 2")
# 
# fig44.errorbar( x   = nominal_values(numDataRCF.QN_s) , y     = nominal_values(numDataRCF.der),\
#               xerr = std_devs(numDataRCF.QN_s)       , yerr  = std_devs(numDataRCF.der),\
#               fmt  = 'o'                                  , label ="rhoCentralFoam")

# fig33.errorbar( x   = nominal_values(largeNozzleData.QN_s) , y     = nominal_values(largeNozzleData.P_vessel_dim),\
#                xerr = std_devs(largeNozzleData.QN_s)       , yerr  = std_devs(largeNozzleData.P_vessel_dim),\
#                fmt  = 'o'                                  , label ="Experimental Large nozzle")

plt.xlabel("Q  NL/min")
plt.title("Derivada dels valors numèrics")
plt.grid() 
axes= plt.gca()
plt.gca().invert_xaxis()
axes.set_xlim([ 0.24 , 0.3 ])
axes.set_ylim([ -7000 , -8000 ])
fig44.legend()

FigureCanvasNbAgg()